link to original notebook
*note: this notebook is missing a signifcant amount of code an will not run on it's own*
https://colab.research.google.com/github/idiap/w2v2-air-traffic/blob/main/src/eval_xlsr_atc_model.ipynb#scrollTo=7ddc6ef1-5a73-40c8-8199-deb3e94bdb2b

In [38]:
import os

from bs4 import BeautifulSoup

import pandas as pd
import matplotlib.pyplot as plt

import torch
from transformers import AutoModelForCTC, Wav2Vec2Processor, AutoProcessor, pipeline
import torchaudio.functional as F

%pip install -r requirements.txt

os.chdir('C:/Users/rowan/OneDrive/Desktop/code/iResearch-2024/DATA')

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'

[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
audioFiles = [f for f in os.listdir() if f.endswith('.wav')]
xmlFiles = [f for f in os.listdir() if f.endswith('.xml')]

transcripts = []

datasetDict = {
	'audio': audioFiles,
	'transcripts': []
}

for i in range(len(xmlFiles)):
	with open(xmlFiles[i], 'r', encoding='utf-8') as file:
		data = file.read()

	Bs_data = BeautifulSoup(data, "xml")
	b_unique = Bs_data.find_all('text')
	for j in b_unique:
		transcripts.append(j.text)
	datasetDict['transcripts'].append(transcripts)
	transcripts = []

df = pd.DataFrame(datasetDict)

df


,audio,transcripts
0,LKPR_RUZYNE_Radar_120_520MHz_20201025_091112.wav,[Oscar Kilo Papa Mike Bravo descend flight lev...
1,LKPR_RUZYNE_Radar_120_520MHz_20201025_120512.wav,[Oscar Kilo Kilo Echo Alfa Praha Radar identif...
2,LKPR_RUZYNE_Radar_120_520MHz_20201025_121325.wav,[Ryanair Seven Three Alpha Hotel turn left hea...
3,LKPR_RUZYNE_Radar_120_520MHz_20201025_130407.wav,[Oscar Kilo Kilo Uniform November proceed dire...
4,LKPR_RUZYNE_Radar_120_520MHz_20201025_140929.wav,[Eurowings Seven Alfa Bravo turn right heading...
...,...,...
555,YSSY_SYDNEY_Tower_120_5MHz_20210505_204538.wav,[[#unnamed]tower[/#unnamed] [#unnamed]good[/#u...
556,YSSY_SYDNEY_Tower_120_5MHz_20210505_214612.wav,[Sydney tower Srilankan Six Zero Six back with...
557,YSSY_SYDNEY_Tower_120_5MHz_20210505_224723.wav,[[#value]runway[/#value] [#value]one[/#value] ...
558,YSSY_SYDNEY_Tower_120_5MHz_20210505_233332.wav,[Sydney tower good day China Eastern Seven Fiv...


In [45]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=False, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]

result = pipe(sample)
print(result["text"])


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
c:\Users\rowan\OneDrive\Desktop\code\iResearch-2024\venv\Lib\site-packages\transformers\models\whisper\generation_whisper.py:480: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


 Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel. Nor is Mr. Quilter's manner less interesting than his matter. He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similes drawn from eating and its results occur most readily to the mind. He has grave doubts whether Sir Frederick Leighton's work is really Greek after all, and can discover in it but little of rocky Ithaca. Linnell's pictures are a sort of Upguards and Adam paintings, and Mason's exquisite idylls are as national as a jingo poem. Mr. Burkett Foster's landscapes smile at one much in the same way that Mr. Carker used to flash his teeth. And Mr. John Collier gives his sitter a cheerful slap on the back before he says, like a shampooer in a Turkish bath, Next man!


In [50]:
from transformers import pipeline

pipe = pipeline("automatic-speech-recognition", model="Jzuluaga/wav2vec2-xls-r-300m-en-atc-uwb-atcc-and-atcosim")

# result = pipe(df['audio'][0])
# result

Some weights of the model checkpoint at Jzuluaga/wav2vec2-xls-r-300m-en-atc-uwb-atcc-and-atcosim were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at Jzuluaga/wav2vec2-xls-r-300m-en-atc-uwb-atcc-and-atcosim and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrization